# Purpose

GakuNin RDM (GRDM) が提供するファイルメタデータを取得する。

## Tested environment

* OS: Windows 11 Pro 25H2
* Python: v3.10.14
    * conda: v25.5.0
* Python module(s):
    * jupyterlab: 4.2.5
    * pydantic: v2.12.4
    * requests: v2.32.3

## Setting

In [ ]:
from getpass import getpass
import os
import requests

In [ ]:
from typing import List, Dict, Any, Optional
from pydantic import BaseModel, Field, field_validator, model_validator


class Creator(BaseModel):
    """
    ER図: Creator
    メタデータ項目 "grdm-file:creators" の値(value)の要素です。
    """
    number: Optional[str] = None
    name_ja: Optional[str] = None
    name_en: Optional[str] = None


class FileMetadata(BaseModel):
    """
    GakuNin RDM のファイルメタデータ全体 (pydantic版)
    """

    # --- 1. item レベルの属性 ---
    # (JSONの 'schema' と 'active' に対応)
    schema_id: str = Field(..., alias="schema")
    active: bool = True

    # --- 2. data レベルの属性 ---
    # 'validation_alias' を使い、JSONキーとPython属性名をマッピング

    data_number: Optional[str] = Field(
        None, validation_alias='grdm-file:data-number')
    title_ja: Optional[str] = Field(
        None, validation_alias='grdm-file:title-ja')
    title_en: Optional[str] = Field(
        None, validation_alias='grdm-file:title-en')
    date_issued_updated: Optional[str] = Field(
        None, validation_alias='grdm-file:date-issued-updated')
    data_description_ja: Optional[str] = Field(
        None, validation_alias='grdm-file:data-description-ja')
    data_description_en: Optional[str] = Field(
        None, validation_alias='grdm-file:data-description-en')
    data_research_field: Optional[str] = Field(
        None, validation_alias='grdm-file:data-research-field')
    data_type: Optional[str] = Field(
        None, validation_alias='grdm-file:data-type')
    file_size: Optional[str] = Field(
        None, validation_alias='grdm-file:file-size')
    data_policy_free: Optional[str] = Field(
        None, validation_alias='grdm-file:data-policy-free')
    data_policy_license: Optional[str] = Field(
        None, validation_alias='grdm-file:data-policy-license')
    data_policy_cite_ja: Optional[str] = Field(
        None, validation_alias='grdm-file:data-policy-cite-ja')
    data_policy_cite_en: Optional[str] = Field(
        None, validation_alias='grdm-file:data-policy-cite-en')
    access_rights: Optional[str] = Field(
        None, validation_alias='grdm-file:access-rights')
    available_date: Optional[str] = Field(
        None, validation_alias='grdm-file:available-date')
    repo_information_ja: Optional[str] = Field(
        None, validation_alias='grdm-file:repo-information-ja')
    repo_information_en: Optional[str] = Field(
        None, validation_alias='grdm-file:repo-information-en')
    repo_url_doi_link: Optional[str] = Field(
        None, validation_alias='grdm-file:repo-url-doi-link')
    hosting_inst_ja: Optional[str] = Field(
        None, validation_alias='grdm-file:hosting-inst-ja')
    hosting_inst_en: Optional[str] = Field(
        None, validation_alias='grdm-file:hosting-inst-en')
    hosting_inst_id: Optional[str] = Field(
        None, validation_alias='grdm-file:hosting-inst-id')
    data_man_type: Optional[str] = Field(
        None, validation_alias='grdm-file:data-man-type')
    data_man_number: Optional[str] = Field(
        None, validation_alias='grdm-file:data-man-number')
    data_man_name_ja: Optional[str] = Field(
        None, validation_alias='grdm-file:data-man-name-ja')
    data_man_name_en: Optional[str] = Field(
        None, validation_alias='grdm-file:data-man-name-en')
    data_man_org_ja: Optional[str] = Field(
        None, validation_alias='grdm-file:data-man-org-ja')
    data_man_org_en: Optional[str] = Field(
        None, validation_alias='grdm-file:data-man-org-en')
    data_man_address_ja: Optional[str] = Field(
        None, validation_alias='grdm-file:data-man-address-ja')
    data_man_address_en: Optional[str] = Field(
        None, validation_alias='grdm-file:data-man-address-en')
    data_man_tel: Optional[str] = Field(
        None, validation_alias='grdm-file:data-man-tel')
    data_man_email: Optional[str] = Field(
        None, validation_alias='grdm-file:data-man-email')
    remarks_ja: Optional[str] = Field(
        None, validation_alias='grdm-file:remarks-ja')
    remarks_en: Optional[str] = Field(
        None, validation_alias='grdm-file:remarks-en')
    metadata_access_rights: Optional[str] = Field(
        None, validation_alias='grdm-file:metadata-access-rights')

    # 1対多の関係
    creators: Optional[List[Creator]] = Field([], validation_alias='grdm-file:creators')

    # --- 3. バリデーション処理 ---

    # (A) 'data' キーのネストを解除し、トップレベルの属性を統合する
    @model_validator(mode='before')
    @classmethod
    def flatten_data_structure(cls, data: Any) -> Any:
        """
        入力辞書の 'data' キーの中身をトップレベルにマージする。
        例: {"schema": "...", "data": {"title_ja": ...}}
            -> {"schema": "...", "title_ja": ...}
        """
        if isinstance(data, dict) and 'data' in data:
            data_content = data.pop('data', {})
            data.update(data_content)
        return data

    # (B) 'creators' フィールドの 'value' ラッパーを解除する
    @field_validator('creators', mode='before')
    @classmethod
    def extract_creators_value(cls, v: Any) -> Any:
        """
        creators の {"value": [...]} 構造からリスト本体を取り出す
        """
        if isinstance(v, dict) and 'value' in v:
            return v['value']
        # 'value' がないか、すでにリストの場合
        if isinstance(v, list):
            return v
        return []

    # (C) 'creators' 以外の全ての 'data' レベル属性の 'value' ラッパーを解除する
    @field_validator(
        'data_number', 'title_ja', 'title_en', 'date_issued_updated',
        'data_description_ja', 'data_description_en', 'data_research_field',
        'data_type', 'file_size', 'data_policy_free', 'data_policy_license',
        'data_policy_cite_ja', 'data_policy_cite_en', 'access_rights',
        'available_date', 'repo_information_ja', 'repo_information_en',
        'repo_url_doi_link', 'hosting_inst_ja', 'hosting_inst_en',
        'hosting_inst_id', 'data_man_type', 'data_man_number', 'data_man_name_ja',
        'data_man_name_en', 'data_man_org_ja', 'data_man_org_en',
        'data_man_address_ja', 'data_man_address_en', 'data_man_tel',
        'data_man_email', 'remarks_ja', 'remarks_en', 'metadata_access_rights',
        mode='before'
    )
    @classmethod
    def extract_value_from_dict(cls, v: Any) -> Any:
        """
        {"value": "..."} 構造から "..." 本体を取り出す
        """
        if isinstance(v, dict) and 'value' in v:
            return v['value']
        return v

    class Config:
        """Pydantic v2 の設定クラス
        """
        # 'validation_alias' で指定したキー（例: 'grdm-file:title-ja'）で
        # データ（'title_ja'）を読み込むことを許可する
        populate_by_name = True

    def get_metadata_by_key(self, key: str) -> Any:
        """
        GakuNin RDMのJSONキー（例: 'grdm-file:title-ja'）または
        Python属性名（例: 'title_ja'）に基づいてメタデータ値を取得します。

        Args:
            key (str): 検索するキー。

        Returns:
            Any: 対応するメタデータ値。見つからない場合は None。
        """

        # Pydanticモデルのフィールド定義 (self.model_fields) をイテレート
        # 'title_ja' のようなPython属性名が attr_name に入る
        for attr_name, field_info in self.model_fields.items():

            # 1. JSONキー (エイリアス) が一致するかチェック
            # (例: field_info.validation_alias が 'grdm-file:title-ja')
            if field_info.validation_alias == key:
                return getattr(self, attr_name)

            # 2. Python属性名が一致するかチェック
            # (例: attr_name が 'title_ja'。'schema' などエイリアスがない属性にも対応)
            if attr_name == key:
                return getattr(self, attr_name)

        # どちらにも一致しない場合
        return None


class FileItem(BaseModel):
    """
    ER図: FileItem
    プロジェクト内の個々のファイルまたはフォルダです。
    """
    path: str
    hash: str
    folder: bool
    urlpath: str
    generated: bool  # JSONデータ例にあったため追加

    # 1つのFileItemは複数のメタデータ(items)を持つことができます
    items: List[FileMetadata]


class ProjectAttributes(BaseModel):
    """
    JSONレスポンスの "attributes" 部分です。
    """
    editable: bool
    features: Dict[str, bool]
    files: List[FileItem]


class ProjectData(BaseModel):
    """
    ER図: Project (JSONの "data" ノードに対応)
    """
    id: str
    type: str
    attributes: ProjectAttributes


class ProjectResponse(BaseModel):
    """
    APIレスポンスのルートオブジェクトです。
    このクラスでJSON全体をパースします。
    """
    data: ProjectData

    # --- データアクセスを容易にするヘルパーメソッド ---

    @property
    def project_id(self) -> str:
        """プロジェクトIDを取得します。"""
        return self.data.id

    @property
    def files(self) -> List[FileItem]:
        """プロジェクト内のすべてのFileItemのリストを取得します。"""
        return self.data.attributes.files

    def find_file_by_path(self, path: str) -> Optional[FileItem]:
        """
        指定されたパスに一致するFileItemを検索します。

        Args:
            path (str): 検索するファイルパス (例: "osfstorage/README.md")

        Returns:
            Optional[FileItem]: 見つかったFileItem、または None
        """
        for item in self.files:
            if item.path == path:
                return item
        return None

    def get_metadata_value(self, file_path: str, field_key: str) -> Optional[Any]:
        """
        特定のファイルのメタデータ項目の「値(value)」を取得します。

        (補足: 複数のスキーマがアクティブな場合、最初に見つかった
         アクティブなスキーマの値を返します)

        Args:
            file_path (str): 検索するファイル/フォルダのパス。
            field_key (str): 取得したいメタデータ項目のキー (例: "grdm-file:title-ja")。

        Returns:
            Optional[Any]: メタデータ項目の値。
                           "grdm-file:creators" の場合は Creator オブジェクトのリスト、
                           それ以外は主に文字列。見つからない場合は None。
        """
        file_item = self.find_file_by_path(file_path)
        if not file_item:
            return None

        # アクティブなメタデータ項目を検索
        target_metadata_item = None
        for meta_item in file_item.items:
            if meta_item.active:
                target_metadata_item = meta_item
                break

        if not target_metadata_item:
            return None

        field_value_obj = target_metadata_item.get_metadata_by_key(field_key)
        if not field_value_obj:
            return None

        raw_value = field_value_obj

        # 'grdm-file:creators' の場合、Creatorオブジェクトのリストにパースする
        if field_key == 'grdm-file:creators' and isinstance(raw_value, list):
            try:
                # パースして Creator オブジェクトのリストを返す
                return [Creator(**item) for item in raw_value]
            except Exception:
                # パース失敗時は生データ(辞書のリスト)を返す
                return raw_value

        # それ以外のキーの場合は、生の値 (主に文字列) を返す
        return raw_value


In [ ]:
# Base url of GRDM API (v1)
GRDM_URL: str = "https://rdm.nii.ac.jp/api/v1"

In [ ]:
# import your personal access token

token: str = os.getenv("GRDM_TOKEN", "")
if not token:
    token = getpass("input your token:")

In [ ]:
# set your project ID

project_id = "input your target GRDM project ID"

## Get file metadata

In [ ]:
endpoint: str = f"{GRDM_URL}/project/{project_id}/metadata/project"
headers = {
    'Accept': 'application/vnd.api+json',
    "Authorization": f"Bearer {token}",
    'Content-Type': "application/json",
}
response = requests.get(
    endpoint, headers=headers,
    timeout=30
)
response.raise_for_status()
print(response)

In [ ]:
project_response = ProjectResponse(**response.json())

project_response.data.id, project_response.data.type

In [ ]:
for node_ in project_response.data.attributes.files:
    node_name = node_.path.split("/")[-1]
    if node_.folder:
        node_name = node_.path.split("/")[-2]
    for metadata in node_.items:
        print(node_name, metadata)